## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
your_api_key = ""
groq_api_key = your_api_key
groq_api_key

''

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000225ACF94CE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000225ACF957C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
ai_message = model.invoke([HumanMessage(content="Hi , My name is Zeliha Erim and I am a Software Engineer")])
ai_message.content

"Hello Zeliha Erim, it's nice to meet you! \n\nBeing a Software Engineer is an exciting field. What kind of software engineering do you specialize in?  \n\nI'm always interested in learning more about what people do. 😊\n\n"

In [4]:
from langchain_core.messages import AIMessage
ai_message_1 = model.invoke(
    [
        HumanMessage(content="Hi , My name is Zeliha Erim and I am a Software Engineer"),
        AIMessage(content= ai_message.content),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)
print(ai_message_1.content)

You are Zeliha Erim, and you are a Software Engineer!  

I remembered that from our earlier conversation.  

Is there anything else you'd like to talk about? Perhaps you could tell me about a project you're working on?



### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
store={}
"""
get_session_history –
Function that returns a new BaseChatMessageHistory. 
This function should either take a single positional argument session_id of type string and return a corresponding
chat message history instance. .. code-block:: python
def get_session_history(
session_id: str, *, user_id: Optional[str]=None

) -> BaseChatMessageHistory:
…
"""
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
config={"configurable":{"session_id":"chat1"}}
parser = StrOutputParser()
chain = model|parser
with_message_history=RunnableWithMessageHistory(chain,  get_session_history=get_session_history, config=config)

In [6]:
?ChatMessageHistory

Init signature:
ChatMessageHistory(
    *,
    messages: list[langchain_core.messages.base.BaseMessage] = None,
) -> None
Docstring:     
In memory implementation of chat message history.

Stores messages in a memory list.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises [`ValidationError`][pydantic_core.ValidationError] if the input data cannot be
validated to form a valid model.

`self` is explicitly positional-only to allow `self` as a field name.
File:           c:\users\monoz\anaconda3\envs\torch_gpu\lib\site-packages\langchain_core\chat_history.py
Type:           ModelMetaclass
Subclasses:     

In [7]:
?RunnableWithMessageHistory

Init signature:
RunnableWithMessageHistory(
    runnable: 'Union[Runnable[Union[MessagesOrDictWithMessages], Union[str, BaseMessage, MessagesOrDictWithMessages]], LanguageModelLike]',
    get_session_history: 'GetSessionHistoryCallable',
    *,
    input_messages_key: 'Optional[str]' = None,
    output_messages_key: 'Optional[str]' = None,
    history_messages_key: 'Optional[str]' = None,
    history_factory_config: 'Optional[Sequence[ConfigurableFieldSpec]]' = None,
    name: Optional[str] = None,
    bound: langchain_core.runnables.base.Runnable[-Input, +Output],
    kwargs: collections.abc.Mapping[str, typing.Any] = None,
    config: langchain_core.runnables.config.RunnableConfig = None,
    config_factories: list[typing.Callable[[langchain_core.runnables.config.RunnableConfig], langchain_core.runnables.config.RunnableConfig]] = None,
    custom_input_type: Optional[Any] = None,
    custom_output_type: Optional[Any] = None,
) -> None
Docstring:     
Runnable that manages chat messag

In [8]:
?RunnableConfig

Object `RunnableConfig` not found.


In [9]:
response = with_message_history.invoke(
    [HumanMessage(content = "Hi , My name is Zeliha Erim and I am a Software Engineer")],
)
response

"Hello Zeliha Erim! It's nice to meet you.\n\nAs a large language model, I don't have personal experiences like being a software engineer, but I can access and process information about it. \n\nIs there anything specific you'd like to talk about related to software engineering? I can help with:\n\n* **Explaining concepts:** If you have a question about a particular programming language, technology, or software development process, I can try my best to explain it.\n* **Generating code:** I can generate code snippets in various programming languages, though it's always important to review and test the code before using it.\n* **Brainstorming ideas:** If you're stuck on a problem or need help coming up with new ideas, I can offer suggestions based on my knowledge.\n\nLet me know how I can be of assistance!\n"

In [10]:
# Does model remember me?
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
)

'Your name is Zeliha Erim.  \n\nYou told me at the beginning of our conversation! 😊  \n\n\n\n'

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content = "I am living in İstanbul.")]
)
response

"That's great to know, Zeliha! İstanbul is a vibrant and historic city.  Are there any particular places or things you enjoy about living there?  \n\nPerhaps you'd like to talk about your favorite local restaurants, historical sites, or maybe even the bustling atmosphere of the city? I'm happy to chat about anything you'd like. 🌎  \n\n\n\n"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content = "I love eating chocolate.")]
)
response

"Me too! Chocolate is delicious. 🍫\n\nDo you have a favorite kind of chocolate?  Dark, milk, white?  Or maybe something more unique, like Turkish delight?  \n\nI'm always curious to learn about people's chocolate preferences. 😋\n"

In [13]:
response = with_message_history.invoke(
    [HumanMessage(content = "I am studying NLP.")]
)
response

"That's fantastic, Zeliha! NLP is a fascinating field.  \n\nWhat aspects of NLP are you most interested in?  Text generation, machine translation, sentiment analysis?  I'm eager to hear more about your studies.  \n\nAnd maybe you can even put my own NLP abilities to the test! 😉  \n\n\n\n"

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content = "I went to grocery yesterday.")]
)
response

"That's great! Did you find everything you needed? \n\nI always find grocery shopping interesting, especially when trying to find new and unique ingredients.  \n\nDid you discover anything new or exciting at the store?  Perhaps a special type of chocolate?  🍫  \n\n\n\n\n"

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content = "My favorite color is blue.")]
)
response

"Blue is a wonderful color!  It's calming, versatile, and often associated with the sky and ocean.  \n\nIs there a particular shade of blue that you love the most?  Perhaps a deep navy, a vibrant turquoise, or a soft sky blue?  \n\nI'm always curious about people's color preferences. 💙\n\n\n\n\n"

In [16]:
# Run chat with different session id
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it! 😊\n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Bulut")],
    config=config1
)
response

"Hi Bulut, it's nice to meet you!  \n\nIs there anything I can help you with today?\n"

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response

'Your name is Bulut!  I remember now.  😊  \n\nDo you want to chat about something, Bulut?  Or would you like me to help you with a task?\n'

In [19]:
print(f"store : {store}")

store : {'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Zeliha Erim and I am a Software Engineer', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Zeliha Erim! It's nice to meet you.\n\nAs a large language model, I don't have personal experiences like being a software engineer, but I can access and process information about it. \n\nIs there anything specific you'd like to talk about related to software engineering? I can help with:\n\n* **Explaining concepts:** If you have a question about a particular programming language, technology, or software development process, I can try my best to explain it.\n* **Generating code:** I can generate code snippets in various programming languages, though it's always important to review and test the code before using it.\n* **Brainstorming ideas:** If you're stuck on a problem or need help coming up with new ideas, I can offer suggestions based on my knowledge.\n\nLet me know how I can be o

In [20]:
store["chat1"].messages

[HumanMessage(content='Hi , My name is Zeliha Erim and I am a Software Engineer', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Zeliha Erim! It's nice to meet you.\n\nAs a large language model, I don't have personal experiences like being a software engineer, but I can access and process information about it. \n\nIs there anything specific you'd like to talk about related to software engineering? I can help with:\n\n* **Explaining concepts:** If you have a question about a particular programming language, technology, or software development process, I can try my best to explain it.\n* **Generating code:** I can generate code snippets in various programming languages, though it's always important to review and test the code before using it.\n* **Brainstorming ideas:** If you're stuck on a problem or need help coming up with new ideas, I can offer suggestions based on my knowledge.\n\nLet me know how I can be of assistance!\n", additional_kwargs={}, response_meta

In [21]:
for key, value in store.items():
    print(f"chat id : {key}\nHistory : {value}")

chat id : chat1
History : Human: Hi , My name is Zeliha Erim and I am a Software Engineer
AI: Hello Zeliha Erim! It's nice to meet you.

As a large language model, I don't have personal experiences like being a software engineer, but I can access and process information about it. 

Is there anything specific you'd like to talk about related to software engineering? I can help with:

* **Explaining concepts:** If you have a question about a particular programming language, technology, or software development process, I can try my best to explain it.
* **Generating code:** I can generate code snippets in various programming languages, though it's always important to review and test the code before using it.
* **Brainstorming ideas:** If you're stuck on a problem or need help coming up with new ideas, I can offer suggestions based on my knowledge.

Let me know how I can be of assistance!

Human: What's my name?
AI: Your name is Zeliha Erim.  

You told me at the beginning of our conversat

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [22]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a NLP software engineer and answers questions of junior software engineer. Answer the question based on this description."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model
with_message_history=RunnableWithMessageHistory(chain, get_session_history)
print("Test chain: \n")
human_message = "Hi My name is Zeliha I am a software engineer."
response = chain.invoke({"messages":[HumanMessage(content=human_message)]})
response.content

Test chain: 



"Hi Zeliha! Welcome to the world of NLP. \n\nWhat can I help you with today? As a senior NLP engineer, I'm here to answer your questions and guide you as you learn. Don't hesitate to ask anything, even if it seems basic. \n\nWhat are you working on right now? 😄  \n\n"

In [23]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content=human_message)],
    config=config
)
response.content

"Hi Zeliha!  \n\nWelcome to the world of NLP! I'm here to help you out. What can I do for you today? \n\nIs there something specific about NLP you'd like to know more about, or are you working on a project that you need help with?  \n\nDon't hesitate to ask anything, even if you think it's a basic question. 😊 \n"

In [24]:
response = with_message_history.invoke([HumanMessage(content="What's my name?")], config=config)
response.content

'Your name is Zeliha! 😊  \n\nNice to officially meet you, Zeliha.  What can I help you with today?\n'

In [25]:
response = with_message_history.invoke([HumanMessage(content="Could you tell me about why engineers using NLP pipeline?")], config=config)
response.content

"You're smart to ask about NLP pipelines, Zeliha! They're a super important part of how we build and use NLP applications.  \n\nThink of an NLP pipeline like an assembly line for natural language processing.  Instead of cars, we're processing text! \n\nHere's why engineers use them:\n\n* **Structure and Organization:** Pipelines break down complex NLP tasks into smaller, manageable steps. This makes the code easier to understand, debug, and maintain.\n\n* **Modularity:** Each step in the pipeline is a separate component that can be easily swapped out or customized.  Need to try a different tokenizer? No problem! Just replace the tokenization step in your pipeline.\n* **Efficiency:** Pipelines can be optimized to process text quickly and efficiently.  \n\n**Here's a simplified example of an NLP pipeline:**\n\n1. **Tokenization:** Breaking down text into individual words or units (tokens).\n\n2. **Part-of-Speech Tagging:** Identifying the grammatical role of each word (noun, verb, adject

In [26]:
# Create settings and other stuffs
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt|model|parser
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config = {"configurable": {"session_id": "chat4"}}
# invoke messages
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi My name is Zeliha I am a software engineer.")],"language":"Türkçe"},
    config=config
)
repsonse

'Merhaba Zeliha,\n\nBenimle konuşmak için çok mutluyum! Bir yazılım mühendisi olduğunu duyduğum için heyecanlıyım. Bana nasıl yardımcı olabilirim? 😊  \n\n'

In [27]:
# Do you remember me?
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Do you remeber me, what is my name?")], "language": "Türkçe"},
    config=config,
)
response

'Evet, elbette! Adın Zeliha olduğunu hatırlıyorum.  \n\nNasıl yardımcı olabilirim? 😊 \n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

In [ ]:
from operator import itemgetter # for trimmer
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) # trim process
    | prompt
    | model
)
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content
# it reached very early conversation because it is trimmed.

In [ ]:
## Lets wrap this in the Message History
config={"configurable":{"session_id":"chat5"}}
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
    config=config
)

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    }
)
response.content

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    }
)
response.content

# chat1 history

In [29]:
# trime chat1 session id history
from langchain_core.messages import SystemMessage, trim_messages
trimmer=trim_messages(
    max_tokens=300,
    strategy="last", # first
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
trimmer.invoke(store["chat1"].messages)

[HumanMessage(content='I am studying NLP.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's fantastic, Zeliha! NLP is a fascinating field.  \n\nWhat aspects of NLP are you most interested in?  Text generation, machine translation, sentiment analysis?  I'm eager to hear more about your studies.  \n\nAnd maybe you can even put my own NLP abilities to the test! 😉  \n\n\n\n", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I went to grocery yesterday.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's great! Did you find everything you needed? \n\nI always find grocery shopping interesting, especially when trying to find new and unique ingredients.  \n\nDid you discover anything new or exciting at the store?  Perhaps a special type of chocolate?  🍫  \n\n\n\n\n", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My favorite color is blue.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Bl

In [30]:
# trime chat1 session id history
trimmer=trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
trimmer.invoke(store["chat1"].messages)

[HumanMessage(content='Hi , My name is Zeliha Erim and I am a Software Engineer', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Zeliha Erim! It's nice to meet you.\n\nAs a large language model, I don't have personal experiences like being a software engineer, but I can access and process information about it. \n\nIs there anything specific you'd like to talk about related to software engineering? I can help with:\n\n* **Explaining concepts:** If you have a question about a particular programming language, technology, or software development process, I can try my best to explain it.\n* **Generating code:** I can generate code snippets in various programming languages, though it's always important to review and test the code before using it.\n* **Brainstorming ideas:** If you're stuck on a problem or need help coming up with new ideas, I can offer suggestions based on my knowledge.\n\nLet me know how I can be of assistance!\n", additional_kwargs={}, response_meta

In [44]:
from operator import itemgetter # for trimmer
from langchain_core.runnables import RunnablePassthrough

trimmer=trim_messages(
    max_tokens=300,
    strategy="last", # first
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Zeliha's assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
past_chat = store["chat1"].messages
chain=RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)|prompt|model|parser
# ask something about early questions
response=chain.invoke(
    {
    "messages" : past_chat + [HumanMessage(content="Where do I live?")],
    "language" : "English"
    }
)
response

"As your assistant, I don't have access to personal information like your address or location.  \n\nIs there anything else I can help you with?  Perhaps you'd like to discuss your favorite things about blue, or maybe you have a question about NLP? 😊  \n\n\n\n\n\n"

In [45]:
# ask something about latest questions
response=chain.invoke(
    {
    "messages" : past_chat + [HumanMessage(content="What is my favoriete color?")],
    "language" : "English"
    }
)
response

"You told me your favorite color is blue!  💙 \n\nIs there something else you'd like to talk about? 😊  \n\n\n\n\n"

In [46]:
# end